# REACT SNIPPETS ⚛️
---

### Use `assert` to fail early

The assert function acts as a safety net. It prevents the app from being built if the NEXT_PUBLIC_API_BASE_URL env variable is missing. So we immediately know that something is wrong before the code is deployed. Note that Next.js ships `dotenv` out of the box

read more: https://dev.to/jkettmann/separate-api-layers-in-react-apps-6-steps-towards-maintainable-code-4n2

In [ ]:
// api/axios.ts

import assert from "assert";
import Axios from "axios";

assert(
  process.env.NEXT_PUBLIC_API_BASE_URL,
  "env variable not set: NEXT_PUBLIC_API_BASE_URL"
);

export const axios = Axios.create({
  baseURL: process.env.NEXT_PUBLIC_API_BASE_URL,
});


// .env

NEXT_PUBLIC_API_BASE_URL=https://prolog-api.profy.dev/v2

---
### decouple auth logic from fetch functions

we can use axios request interceptors to remove headers from fetch request

```
axios.get("/endpoint", {
    params: { page, status: 'open'},
    signal: options.signla,
    headers: {Authoriztion: "my-access-token"} ❗️
})
```

In [ ]:
// api/axios.ts

import assert from "assert";
import Axios, { AxiosRequestConfig } from "axios";

assert(
  process.env.NEXT_PUBLIC_API_BASE_URL,
  "env variable not set: NEXT_PUBLIC_API_BASE_URL"
);

assert(
  process.env.NEXT_PUBLIC_API_TOKEN,
  "env variable not set: NEXT_PUBLIC_API_TOKEN"
);

function authRequestInterceptor(config: AxiosRequestConfig) {
  config.headers.authorization = process.env.NEXT_PUBLIC_API_TOKEN;
  return config;
}

export const axios = Axios.create({
  baseURL: process.env.NEXT_PUBLIC_API_BASE_URL,
});

axios.interceptors.request.use(authRequestInterceptor);

We already achieved a good separation between the fetch functions and the query hooks. As mentioned, the query hooks don’t have any knowledge about Axios being used or about request details like the endpoints.

But the fetch functions themselves aren’t decoupled from Axios yet. We directly export the Axios client and use it in the fetch functions.

In [ ]:
// api/axios.ts

export const axios = Axios.create({
    baseURL: process.env.NEXT_PUBLIC_API_BASE_URL,
  });
  

// api/issues.ts

import { axios } from "./axios";

export async function getIssues(...) {
  const { data } = await axios.get(ENDPOINT, {
    params: { page, ...filters },
    signal: options?.signal,
  });
  return data;
}

If we wanted to replace Axios with something else we’d have to adjust all the fetch functions as well.

To create a further isolation layer we can simply wrap the axios client and only expose its methods indirectly.

In [ ]:
// api/api-client.ts

const axios = Axios.create({
    baseURL: process.env.NEXT_PUBLIC_API_BASE_URL,
  });
  
  export const apiClient = {
    get: (route, config) =>
      axios.get(route, { signal: config?.signal, params: config?.params }),
    post: (route, data, config) =>
      axios.post(route, data, { signal: config?.signal }),
    put: (route, data, config) =>
      axios.put(route, data, { signal: config?.signal }),
    patch: (route, data, config) =>
      axios.patch(route, data, { signal: config?.signal }),
  };

Note that we don’t pass the config object directly to axios. Otherwise, the fetch functions could use any config option that Axios supports. And that again would couple them to Axios.

The fetch function basically would stay the same.

In [ ]:
// api/issues.ts

import { apiClient } from "./api-client";

export async function getIssues(...) {
  const { data } = await apiClient.get(ENDPOINT, {
    params: { page, ...filters },
    signal: options?.signal,
  });
  return data;
}

React tip: If state values are related, and must be kept in sync, create the "derived" value on render.

Example: If you have data and filteredData, create filteredData on render.

Doing so eliminates a useEffect, and needless syncing code. This reduces the chance of bugs.

In [ ]:
//! AVOID: routes and filteredRoutes need to be kept in sync 👎
const [routes, setRoutes] = useState([])
const [filteredRoutes, setFilteredRoutes] = useState([])
const [search, setSearch] = useState("")

// Using useEffect to sync related stated
useEffect(() => {
    setFilteredRoutes(routes.filter((route) => {
        // filtering logic here
    }))
}, [search])

//* PREFERED: Derive related state on render. This eliminates the need for useEffect 👍
const [routes2, setRoutes2] = useState([])
const [search2, setSearch2] = useState("")

// Derive filtered routes on each render. 
// If this is expensive, memoize it.
const filteredRoutes2 = routes.filter((route) => {
            // filtering logic here
})

![](derived_state_tip.png)

---

### Mediator Patter in React

https://itnext.io/improve-state-management-in-react-with-mediator-design-pattern-db632cbe6475

Mediator Pattern to Decouple State Management in React
Managing the state of multiple components in a React application can be a challenging task, but one solution that has gained popularity is the use of the Mediator pattern. By creating a mediator component that manages the application state and handles communication between components, the Mediator pattern promotes loose coupling, scalability, reusability, maintainability, and testability.

Rather than having components communicate directly with each other, the mediator component acts as a central hub that facilitates communication between components. This approach allows for better separation of concerns, making it easier to add new features or modify existing ones without disrupting the entire application.

Using the Mediator pattern as a state management solution in React applications also promotes modularization, making it easier to reuse components in other parts of the application or even in other applications.

Here are the general steps to implement the Mediator pattern as a state management solution in a React application:

- Identify the state that needs to be managed: Identify the state that needs to be managed and which components need access the state.
- Create a Mediator component: Create a Mediator component that will manage the state and handle communication between the components. The Mediator component will store the state and update it when necessary.
- Pass the Mediator as a prop: Pass the Mediator component as a prop to the components that need access to the state.
- Implement event handlers: Implement event handlers in the components that need to update the state. These event handlers should trigger a function in the Mediator component that updates the state.
- Use the state: Use the state stored in the Mediator component in the components that need access to it.


Let’s take a look at an example code snippet that demonstrates how to implement the Mediator pattern as a state management solution in a React application:

In [ ]:
// Mediator component
const Mediator = ({ children }) => {
  const [state, setState] = useState({ count: 0 });

  const incrementCount = () => {
    setState({ count: state.count + 1 });
  };

  const decrementCount = () => {
    setState({ count: state.count - 1 });
  };

  // Pass state and event handlers as props to children components
  return children({ count: state.count, incrementCount, decrementCount });
};

// Child component
const Counter = ({ count, incrementCount, decrementCount }) => {
  return (
    <div>
      <p>Count: {count}</p>
      <button onClick={incrementCount}>Increment</button>
      <button onClick={decrementCount}>Decrement</button>
    </div>
  );
};

// Parent component
const App = () => {
  return (
    <Mediator>
      {({ count, incrementCount, decrementCount }) => (
        <Counter count={count} incrementCount={incrementCount} decrementCount={decrementCount} />
      )}
    </Mediator>
  );
};

Comparing the Mediator Pattern to Context API as State Management
While both the Mediator Pattern and Context API can be used for state management in a React application, there are some key differences between them.

The Mediator Pattern promotes loose coupling between components, while the Context API promotes tighter coupling between components. In the Mediator Pattern, components communicate through a Mediator object, while in the Context API, components directly access the shared state.

At the same time, the Mediator Pattern can be more complex to implement than the Context API, as it requires creating a custom hook and managing state within the Mediator object. The Context API is simpler to implement, as it only requires creating a Context object and providing a Provider component.

Both the Mediator Pattern and Context API have their strengths and weaknesses, and the choice of which to use will depend on the application's specific requirements.

The Mediator Pattern is well-suited for applications with complex state management requirements and a large number of components that need to communicate with each other. It can provide a more modular and flexible approach to state management, allowing for easier maintenance and extensibility.

The Context API is better suited for applications with simpler state management requirements and a smaller number of components. It can provide a simpler and more straightforward approach to state management, allowing for faster development and easier code maintenance.

It’s important to understand the differences between the Mediator Pattern and Context API for state management in React. The Mediator Pattern offers greater flexibility for complex state management, while the Context API provides a simpler approach for applications with fewer components. Ultimately, the choice of which to use depends on the application's specific requirements.

---
### JWT Authentication with React Router
https://dev.to/sanjayttg/jwt-authentication-in-react-with-react-router-1d03

In [ ]:
import axios from "axios";
import { createContext, useContext, useEffect, useMemo, useState } from "react";

const AuthContext = createContext();

const AuthProvider = ({ children }) => {
  // State to hold the authentication token
  const [token, setToken_] = useState(localStorage.getItem("token"));

  // Function to set the authentication token
  const setToken = (newToken) => {
    setToken_(newToken);
  };

  useEffect(() => {
    if (token) {
      axios.defaults.headers.common["Authorization"] = "Bearer " + token;
      localStorage.setItem('token',token);
    } else {
      delete axios.defaults.headers.common["Authorization"];
      localStorage.removeItem('token')
    }
  }, [token]);

  // Memoized value of the authentication context
  const contextValue = useMemo(
    () => ({
      token,
      setToken,
    }),
    [token]
  );

  // Provide the authentication context to the children components
  return (
    <AuthContext.Provider value={contextValue}>{children}</AuthContext.Provider>
  );
};

export const useAuth = () => {
  return useContext(AuthContext);
};

export default AuthProvider;